In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/ADMISSIONS.csv')
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [3]:
df.shape

(58976, 19)

In [4]:
df_subjectid = pd.read_csv('../data/Diabete_subjectID.csv')
df_subjectid.drop(columns='Unnamed: 0',inplace=True)
diabetes_id = df_subjectid['SUBJECT_ID']
len(diabetes_id)

7370

In [5]:
diabetes_id = list(diabetes_id)

In [6]:
# Create a column to show diabete status
df['diabete'] = df['SUBJECT_ID'].apply(lambda x: 1 if x in diabetes_id else 0)

In [7]:
# All diabete patients are included in this dataframe
len(df[df['diabete']==1]['SUBJECT_ID'].unique())

7370

In [8]:
df_diabetes = df.loc[df['diabete']==1, ['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DEATHTIME']]
df_diabetes.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN
7,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,NaN
14,35,166707,2122-02-10 11:15:00,2122-02-20 15:30:00,NaN
18,37,188670,2183-08-21 16:48:00,2183-08-26 18:54:00,NaN
22,357,174486,2197-12-06 07:13:00,2198-01-03 14:00:00,NaN


In [9]:
df_diabetes.shape

(11183, 5)

In [10]:
df_diabetes['ADMITTIME'] = pd.to_datetime(df_diabetes['ADMITTIME']).dt.date
df_diabetes['DISCHTIME'] = pd.to_datetime(df_diabetes['DISCHTIME']).dt.date
df_diabetes['DEATHTIME'] = pd.to_datetime(df_diabetes['DEATHTIME']).dt.date

In [11]:
df_diabetes['NEXT_ADMITTIME'] = df_diabetes.groupby('SUBJECT_ID').ADMITTIME.shift(-1)

In [12]:
df_diabetes.head(10)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,NEXT_ADMITTIME
3,24,161859,2139-06-06,2139-06-09,NaT,NaN
7,28,162569,2177-09-01,2177-09-06,NaT,NaN
14,35,166707,2122-02-10,2122-02-20,NaT,NaN
18,37,188670,2183-08-21,2183-08-26,NaT,NaN
22,357,174486,2197-12-06,2198-01-03,NaT,2198-08-02
23,357,145674,2198-08-02,2198-10-26,NaT,2198-11-01
24,357,122609,2198-11-01,2198-11-14,NaT,2199-10-20
25,357,101651,2199-10-20,2199-10-23,NaT,2199-12-21
26,357,117876,2199-12-21,2200-01-18,NaT,NaN
28,359,144265,2176-06-30,2176-07-03,NaT,NaN


In [13]:
df_diabetes.shape

(11183, 6)

In [14]:
df_diabetes['READMIT_DAYS'] = df_diabetes['NEXT_ADMITTIME'] - df_diabetes['DISCHTIME']

In [15]:
df_diabetes.shape

(11183, 7)

In [16]:
df_diabetes.drop_duplicates('SUBJECT_ID', keep='first',inplace=True)

In [17]:
df_diabetes[df_diabetes['READMIT_DAYS']<'30 days']['DEATHTIME'].value_counts()

2111-05-09    1
2127-09-15    1
2116-08-15    1
2113-01-30    1
2131-02-12    1
Name: DEATHTIME, dtype: int64

In [18]:
df_diabetes = df_diabetes[df_diabetes['DEATHTIME'].isnull()]

In [19]:
# Number of patients diagnose as diabetes who's not dead
df_diabetes.shape

(6601, 7)

In [23]:
id_list = pd.read_csv('../data/discharge_id_list.csv')

In [30]:
id_list=list(id_list['0'])

In [22]:
df_diabetes.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,NEXT_ADMITTIME,READMIT_DAYS
3,24,161859,2139-06-06,2139-06-09,NaT,NaN,NaT
7,28,162569,2177-09-01,2177-09-06,NaT,NaN,NaT
14,35,166707,2122-02-10,2122-02-20,NaT,NaN,NaT
18,37,188670,2183-08-21,2183-08-26,NaT,NaN,NaT
22,357,174486,2197-12-06,2198-01-03,NaT,2198-08-02,211 days


In [51]:
df_dia = df_diabetes[df_diabetes['HADM_ID'].map(lambda x: x in id_list)]

In [52]:
df_dia.shape

(6483, 7)

In [63]:
df_dia.sort_values('HADM_ID')

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,NEXT_ADMITTIME,READMIT_DAYS
843,533,100009,2162-05-16,2162-05-21,NaT,NaN,99 days
41003,51495,100034,2164-04-23,2164-04-27,NaT,NaN,99 days
1673,1569,100045,2176-02-05,2176-02-15,NaT,2177-05-15,455 days
24027,20777,100059,2198-02-01,2198-02-19,NaT,NaN,99 days
53131,82091,100099,2118-05-07,2118-05-19,NaT,2118-07-30,72 days
...,...,...,...,...,...,...,...
13439,9062,199923,2175-03-13,2175-03-17,NaT,NaN,99 days
27230,22711,199952,2134-03-07,2134-04-06,NaT,NaN,99 days
5378,5478,199963,2101-08-23,2101-09-08,NaT,NaN,99 days
37652,42131,199967,2184-07-31,2184-08-10,NaT,NaN,99 days


In [62]:
df_dia['READMIT_DAYS'].fillna((pd.to_timedelta('99 days')),inplace=True)

In [66]:
import numpy as np
df_dia['label'] = df_dia['READMIT_DAYS'].map(lambda x: (1 if x < np.timedelta64(30,'D') else 0))

/Users/susanchen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
df_dia.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,NEXT_ADMITTIME,READMIT_DAYS,label
3,24,161859,2139-06-06,2139-06-09,NaT,NaN,99 days,0
7,28,162569,2177-09-01,2177-09-06,NaT,NaN,99 days,0
14,35,166707,2122-02-10,2122-02-20,NaT,NaN,99 days,0
18,37,188670,2183-08-21,2183-08-26,NaT,NaN,99 days,0
22,357,174486,2197-12-06,2198-01-03,NaT,2198-08-02,211 days,0


In [68]:
df_dia['label'].value_counts()

0    6008
1     475
Name: label, dtype: int64

In [71]:
df_dia[['HADM_ID','label']].to_csv('../data/MIMIC_id_label.csv')

In [40]:
len(df_dia[df_dia['READMIT_DAYS']<'30 days'])

475

In [20]:
# Number of patients readmit within 30 days
len(df_diabetes[df_diabetes['READMIT_DAYS']<'30 days'])

480

In [21]:
df_diabetes['HADM_ID'].to_csv('../data/addmision_id.csv')

/Users/susanchen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
